# In Detail about TF-Files

* The .ckpt file is the old version output of saver.save(sess), which is the equivalent of your .ckpt-data (see below)

* The .ckpt-data contains the values for all the variables, without the structure. To restore a model in python, you'll usually use the meta and data files with (but you can also use the .pb file):

> saver = tf.train.import_meta_graph(path_to_ckpt_meta)

> saver.restore(sess, path_to_ckpt_data)

* The "checkpoint" file is only here to tell some TF functions which is the latest checkpoint file.

* The .ckpt-meta contains the metagraph, i.e. the structure of your computation graph, without the values of the variables (basically what you can see in tensorboard/graph).


* The .ckpt-index, I guess it's some kind of index needed internally to map the two previous files correctly. Anyway it's not really necessary usually, you can restore a model with only .ckpt-meta and .ckpt-data.

* The .pb file can save your whole graph (meta + data). To load and use (but not train) a graph. Usually use it, created with freeze_graph, which creates the .pb file from the meta and data. Be careful, (at least in previous TF versions and for some people) the py function provided by freeze_graph did not work properly, so you'd have to use the script version. 

# Convert Dataset to TFRecord

To avoid confusion, a TensorFlow 'example' is a normalized data format for storing data for training and inference purposes. It contains a key-value store where each key string maps to a feature message. These feature messages can be things like a packed byte list, float list, int64 list. 

In [4]:
''' when want to change a other models, Let's do following:
    1. Access to Model Zoo: https://github.com/tensorflow/models/tree/master/research/slim
    2. Download Model
    3. Change TFRecord files 's path  (Dataset Information - line 1)
              checkpoint file 's path (Dataset Information - line 7)
              label txt file 's path (Dataset Information - line 16)
    4. Get inception_v3.py in TF - slim folder and import it as (***)
      ---> from inception_v3 import inception_v3, inception_v3_arg_scope
    5. Called as inception_v3, inception_v3_arg_scope must change all of them in Run()
'''
import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging
import inception_preprocessing
from inception_v3 import inception_v3, inception_v3_arg_scope # (***)
import os
import time
slim = tf.contrib.slim

# Dataset Information
### State dataset directory where the TFRecord files are located

In [13]:
dataset_dir = 'flowers'

# State where ur log file is at. If it doesn't exist, create it
log_dir = 'log'

#State where your checkpoint file is
checkpoint_file = 'checkpoint/inception_v3.ckpt'

#State the image size you're resizing your images to. We will use the default inception size of 299.
image_size = 299

#State the num of classes to predict
num_classes = 5

#State the labels file and read
label_file = 'flowers/labels.txt'
labels = open(label_file, 'r')

# Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    print(label)
    string_name = string_name[:-1] # Remove newline
    labels_to_name[int(label)] = string_name

#Create the file pattern of your 
#TFRecord files so that it could be recognized later on
file_pattern = 'flower_%s_*.tfrecord'

items_to_descriptions = {
    'image': 'A 3-channel RGB coloured flower image that is either tulips, sunflowers, roses, dandelion, or daisy.',
    'label': 'A label that is as such -- 0:daisy, 1:dandelion, 2:roses, 3:sunflowers, 4:tulips'
}

0
1
2
3
4


# Training 

In [8]:
num_epochs = 70
batch_size = 10

#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 0.0002
learning_rate_decay_factor = 0.7
num_epochs_before_decay = 2

# Creating A DataSet Object

This function get TFrecord file to generate Dataset

In [9]:
'''INPUTS:
    - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
    - dataset_dir(str): the dataset directory where the tfrecord files are located
    - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data
    - file_pattern_for_counting(str): the string name to identify your tfrecord files for counting
    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.'''
    
        
def get_split(split_name, dataset_dir, file_pattern=file_pattern, file_pattern_for_counting='flower'):
    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError('The split_name %s is bit recognized.\
                        Please input either train or validation\
                        as the split_name' % (split_name))
    
    #Create the full path for a general file_pattern to locate the tfrecord_files.
    file_pattern_path = os.path.join(dataset_dir, 
                                     file_pattern % (split_name))
    
    # Count the total number of examples in all of these shard
    num_samples = 0
    file_pattern_for_counting = file_pattern_for_counting  + '_' + split_name
    tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.startswith(file_pattern_for_counting)]
    
    for tfrecord_file in tfrecords_to_count:
        for record in tf.python_io.tf_record_iterator(tfrecord_file):
            num_samples += 1
    
    # Create a reader , which must be a TFRecord reader in this case
    reader = tf.TFRecordReader
    
    #Create the keys_to_features dictionary for the decoder
    key_to_features = {
        'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
        'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
        'image/class/label': tf.FixedLenFeature(
            [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }
    
    #Create the items_to_handlers dictionary for the decoder
    items_to_handlers = {
        'image': slim.tfexample_decoder.Image(),
        'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }
    
    # Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(key_to_features, items_to_handlers)
    
    # Create the labels to name file
    labels_to_name_dict = labels_to_name
    
    # Actually create the dataset
    dataset = slim.dataset.Dataset(
            data_sources= file_pattern_path,
            decoder= decoder,
            reader= reader,
            num_reader = 4,
            num_samples = num_samples,
            num_classes = num_classes,
            labels_to_name= labels_to_name_dict,
            items_to_descriptions= items_to_descriptions
    )
    
    return dataset



# Load Batch Functions
- tf.expand_dims will expand the 3D tensor from a [height, width, channels] shape to [1, height, width, channels] shape


- tf.squeeze will simply eliminate all the dimensions with the number ‘1’, which brings the raw_image back to the same 3D shape after reshaping.

## From Dataset get from get_split function, we will load images, _ , labels

In [10]:
def load_batch(dataset, batch_size, height=image_size, width=image_size, is_training=True):
    '''
    Loads a batch for training.
    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing
    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).
    '''
    
    # Create the data Provider object
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
                dataset,
                common_queue_capacity= 24 + 3 * batch_size,
                common_queue_min = 24)
    
    #obtain the raw image using the get method
    raw_image, label = data_provider.get(['image','label'])
    
    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_preprocessing.preprocess_image(raw_image, height, width, is_training)
    
    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height,width])
    
    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_image, labels = tf.train.batch(
                    [image, raw_image, label],
                    batch_size = batch_size,
                    num_threads = 4,
                    capacity = 3 * batch_size,
                    allow_smaller_final_batch = True)
    
    return images, raw_image, labels


        

# TRAINING PROCESS 

### Logits Layer

更にもう一度 tf.layers.dense を呼び出すことにより出力層を定義します、今回は活性化関数なしでです :

1. Compute logits (1 per class).
2. logits = tf.layers.dense(net, params['n_classes'], activation=None)

ここで、net は最後の隠れ層を表します。従って、層の完全な層は今では次のように接続されています :


![title](Imagedoc/logits.png)


出力層を定義するとき、units パラメータは出力の数を指定します。それで、units を params[‘n_classes’] に設定することで、モデルはクラス毎に一つの出力値を生成します。出力ベクトルの各要素は、それぞれアイリスの関連するクラス: セトサ, バージカラー, バージニカのために計算された、スコア、あるいは「ロジット」を含むでしょう。

後で、これらのロジットは tf.nn.softmax 関数により確率へと変換されるでしょう。


### Speed up during the training

- To increase the speed, one way is to also stop calling the summaries so often every 10 steps. Maybe calling summaries every 30 steps is a good interval as well


In [21]:
def run():
    #Create the log directory here. Must be done here otherwise import will activate this unneededly.
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    
    #Now we start to construct the graph and build our model
    with tf.Graph().as_default() as graphs:
        # Set the verbosity to INFO level
        tf.logging.set_verbosity(tf.logging.INFO)
        
        # First create the dataset and load one batch
        dataset = get_split('train', dataset_dir, file_pattern)
        images, _, labels = load_batch(dataset, batch_size=batch_size)
        
        # Know the num steps to take before decaying the learning rate and batches per epoch
        num_batches_per_epoch = int(dataset.num_samples / batch_size)
        # One step is one batch processed
        num_steps_per_epoch = num_batches_per_epoch
        decay_steps = int(num_epochs_before_decay * num_steps_per_epoch)
        
        #------------------------------------------------------------------------create the model inference
        with slim.arg_scope(inception_v3_arg_scope()):
            logits, end_points = inception_v3(images, num_classes=dataset.num_classes,is_training=True)
            
        # Define the scopes that want to exclude for restoration
        exclude = ['InceptionV3/Logits', 'InceptionV3/AuxLogits']
        variables_to_restore = slim.get_variables_to_restore(exclude = exclude)
        
        #Perform one-hot-encoding of the labels (Try one-hot-encoding within the load_batch function!)
        one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
        
        #Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced with checks
        loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
        #obtain the regularization losses as well
        total_loss = tf.losses.get_total_loss()
        
        #Create the global step for monitoring the learning rate and training
        global_step = get_or_create_global_step()
        
        #Define your exponentially decaying learning rate
        lr = tf.train.exponential_decay(
            learning_rate = initial_learning_rate,
            global_step = global_step,
            decay_steps = decay_steps,
            decay_rate = learning_rate_decay_factor,
            staircase = True)
        
        
        #Now we can define the optimizer that takes on the learning rate
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        
        #Create the train_op
        train_op = slim.learning.create_train_op(total_loss, optimizer)
        
        #State the metrics that you want to predict.
        #We get a predictions that is not one_hot_encoded.
        predictions = tf.arg_max(end_points['Predictions'], 1)
        probabilities = end_points['Predictions']                                    
        accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
        metrics_op = tf.group(accuracy_update, probabilities)
        
        
        #--------------------------------------------------------------------------------------------------
        #Now finally create all the summaries you need 
        # to monitor and group them into one summary op.
        tf.summary.scalar('losses/Total_Loss', total_loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.scalar('learning_rate', lr)
        my_summary_op = tf.summary.merge_all()
        
        #Now we need to create a training step function that 
        #runs both the train_op, metrics_op and updates the global_step concurrently.
        def train_step(sess, train_op, global_step):
            '''
            Simply runs a session for the three arguments provided and gives a logging on the time elapsed for each global step
            '''
            #Check the time for each sess run
            start_time = time.time()
            total_loss, global_step_count, _ = sess.run([train_op, global_step, metrics_op])
            tim_elapsed = time.time() - start_time
            
            #Run the logging to print some results
            logging.info('global step %s: loss: %.4f (%.2f sec/step)', global_step_count, total_loss, tim_elapsed)

            return total_loss, global_step_count
        
        #Now we create a saver function that actually restores 
        #the variables from a checkpoint file in a sess
        saver = tf.train.Saver(variables_to_restore)
        def restore_fn(sess):
            return saver.restore(sess, checkpoint_file)
        
        #Define your supervisor for running a managed session. 
        #Do not run the summary_op automatically or else it will consume too much memory
        sv = tf.train.Supervisor(logdir = log_dir, summary_op = None, init_fn = restore_fn)
        
        
        #-------------------------------------------------------------------------------Run the managed session
        with sv.managed_session() as sess:
            for step in range(num_steps_per_epoch * num_epochs): 
                #At the start of every epoch, show the vital information:
                if step % num_batches_per_epoch == 0:
                    logging.info('Epoch %s/%s', step/num_batches_per_epoch + 1, num_epochs)
                    learning_rate_value, accuracy_value = sess.run([lr, accuracy])
                    logging.info('Current Learning Rate: %s', learning_rate_value)
                    logging.info('Current Streaming Accuracy: %s', accuracy_value)

                    # optionally, print your logits and predictions for a sanity check that things are going fine.
                    logits_value, probabilities_value, predictions_value, labels_value = sess.run([logits, probabilities, predictions, labels])
                    print ('logits: \n', logits_value)
                    print ('Probabilities: \n', probabilities_value)
                    print ('predictions: \n', predictions_value)
                    print ('Labels:\n:', labels_value)

                #Log the summaries every 10 step.
                if step % 10 == 0:
                    loss, _ = train_step(sess, train_op, sv.global_step)
                    summaries = sess.run(my_summary_op)
                    sv.summary_computed(sess, summaries)
                    
                #If not, simply run the training step
                else:
                    loss, _ = train_step(sess, train_op, sv.global_step)

            #We log the final training loss and accuracy
            logging.info('Final Loss: %s', loss)
            logging.info('Final Accuracy: %s', sess.run(accuracy))

            #Once all the training has been done, save the log files and checkpoint model
            logging.info('Finished training! Saving model to disk now.')
            # saver.save(sess, "./flowers_model.ckpt")
            sv.saver.save(sess, sv.save_path, global_step = sv.global_step)

In [22]:
run()

INFO:tensorflow:Restoring parameters from checkpoint/inception_v3.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path log/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Epoch 1.0/70
INFO:tensorflow:Current Learning Rate: 0.0002
INFO:tensorflow:Current Streaming Accuracy: 0.0
logits: 
 [[-0.94096607  0.87667084  0.38831222  0.06610719  0.8653086 ]
 [ 0.23982334  0.04781993  0.24227962  1.5000325   0.20631668]
 [-0.41513982  0.24249746  0.21093586  0.02104389 -0.33769575]
 [-0.40196353 -0.6629572  -0.08471175 -0.05873361  0.7997777 ]
 [-0.30936593 -0.22257312  0.3485444   1.0280843   0.00794045]
 [-0.67065936  0.21080792  0.48385385  0.12617484  0.30732483]
 [-0.60658395  0.7319138   0.65256804  0.82440096  0.39626646]
 [-1.168242   -0.12242656  0.33964086 -0.6984473   0.636929  ]
 [ 0.5039619  -0.18944478  0.4803

INFO:tensorflow:global step 106: loss: 0.5802 (0.19 sec/step)
INFO:tensorflow:global step 107: loss: 0.7907 (0.19 sec/step)
INFO:tensorflow:global step 108: loss: 0.6718 (0.19 sec/step)
INFO:tensorflow:global step 109: loss: 0.7254 (0.18 sec/step)
INFO:tensorflow:global step 110: loss: 0.8035 (0.19 sec/step)
INFO:tensorflow:global step 111: loss: 1.7432 (0.18 sec/step)
INFO:tensorflow:global step 112: loss: 1.5514 (0.19 sec/step)
INFO:tensorflow:global step 113: loss: 0.4707 (0.19 sec/step)
INFO:tensorflow:global step 114: loss: 0.9689 (0.19 sec/step)
INFO:tensorflow:global step 115: loss: 0.5857 (0.19 sec/step)
INFO:tensorflow:global step 116: loss: 1.1603 (0.20 sec/step)
INFO:tensorflow:global step 117: loss: 0.7276 (0.19 sec/step)
INFO:tensorflow:global step 118: loss: 1.1207 (0.18 sec/step)
INFO:tensorflow:global step 119: loss: 0.3834 (0.19 sec/step)
INFO:tensorflow:global step 120: loss: 0.5167 (0.19 sec/step)
INFO:tensorflow:global step 121: loss: 0.6546 (0.18 sec/step)
INFO:ten

INFO:tensorflow:global step 239: loss: 0.8271 (0.17 sec/step)
INFO:tensorflow:global step 240: loss: 0.7481 (0.17 sec/step)
INFO:tensorflow:global step 241: loss: 0.9584 (0.17 sec/step)
INFO:tensorflow:global step 242: loss: 1.0497 (0.17 sec/step)
INFO:tensorflow:global step 243: loss: 0.8607 (0.17 sec/step)
INFO:tensorflow:global step 244: loss: 1.0630 (0.17 sec/step)
INFO:tensorflow:global step 245: loss: 0.5357 (0.18 sec/step)
INFO:tensorflow:global step 246: loss: 0.5466 (0.18 sec/step)
INFO:tensorflow:global step 247: loss: 0.8493 (0.17 sec/step)
INFO:tensorflow:global step 248: loss: 0.3410 (0.17 sec/step)
INFO:tensorflow:global step 249: loss: 0.9884 (0.18 sec/step)
INFO:tensorflow:global step 250: loss: 0.6909 (0.17 sec/step)
INFO:tensorflow:global step 251: loss: 0.3610 (0.17 sec/step)
INFO:tensorflow:global step 252: loss: 0.3667 (0.18 sec/step)
INFO:tensorflow:global step 253: loss: 1.2103 (0.18 sec/step)
INFO:tensorflow:global step 254: loss: 0.7416 (0.18 sec/step)
INFO:ten

INFO:tensorflow:global step 345: loss: 0.8600 (0.17 sec/step)
INFO:tensorflow:global step 346: loss: 0.4289 (0.17 sec/step)
INFO:tensorflow:global step 347: loss: 1.8168 (0.17 sec/step)
INFO:tensorflow:global step 348: loss: 0.4028 (0.17 sec/step)
INFO:tensorflow:global step 349: loss: 0.3415 (0.17 sec/step)
INFO:tensorflow:global step 350: loss: 0.7061 (0.17 sec/step)
INFO:tensorflow:global step 351: loss: 0.6020 (0.17 sec/step)
INFO:tensorflow:global step 352: loss: 1.0688 (0.17 sec/step)
INFO:tensorflow:global step 353: loss: 0.5791 (0.17 sec/step)
INFO:tensorflow:global step 354: loss: 0.6605 (0.17 sec/step)
INFO:tensorflow:global step 355: loss: 0.8246 (0.18 sec/step)
INFO:tensorflow:global step 356: loss: 0.5819 (0.17 sec/step)
INFO:tensorflow:global step 357: loss: 1.1927 (0.17 sec/step)
INFO:tensorflow:global step 358: loss: 0.6712 (0.18 sec/step)
INFO:tensorflow:global step 359: loss: 0.3414 (0.17 sec/step)
INFO:tensorflow:global step 360: loss: 0.7876 (0.17 sec/step)
INFO:ten

INFO:tensorflow:global step 478: loss: 1.2729 (0.17 sec/step)
INFO:tensorflow:global step 479: loss: 0.8571 (0.20 sec/step)
INFO:tensorflow:global step 480: loss: 0.8361 (0.18 sec/step)
INFO:tensorflow:global step 481: loss: 0.6670 (0.18 sec/step)
INFO:tensorflow:global step 482: loss: 0.7141 (0.17 sec/step)
INFO:tensorflow:global step 483: loss: 0.4674 (0.17 sec/step)
INFO:tensorflow:global step 484: loss: 0.6275 (0.17 sec/step)
INFO:tensorflow:global step 485: loss: 0.8212 (0.17 sec/step)
INFO:tensorflow:global step 486: loss: 0.3403 (0.17 sec/step)
INFO:tensorflow:global step 487: loss: 0.6418 (0.17 sec/step)
INFO:tensorflow:global step 488: loss: 0.5448 (0.17 sec/step)
INFO:tensorflow:global step 489: loss: 0.4481 (0.17 sec/step)
INFO:tensorflow:global step 490: loss: 0.9562 (0.17 sec/step)
INFO:tensorflow:global step 491: loss: 1.0274 (0.18 sec/step)
INFO:tensorflow:global step 492: loss: 0.5959 (0.18 sec/step)
INFO:tensorflow:global step 493: loss: 1.5181 (0.17 sec/step)
INFO:ten

...

INFO:tensorflow:global step 17414: loss: 0.5789 (0.18 sec/step)
INFO:tensorflow:global step 17415: loss: 0.2363 (0.19 sec/step)
INFO:tensorflow:global step 17416: loss: 0.2395 (0.18 sec/step)
INFO:tensorflow:global step 17417: loss: 0.3448 (0.17 sec/step)
INFO:tensorflow:global step 17418: loss: 0.6505 (0.17 sec/step)
INFO:tensorflow:global step 17419: loss: 0.7780 (0.17 sec/step)
INFO:tensorflow:global step 17420: loss: 0.2611 (0.17 sec/step)
INFO:tensorflow:global step 17421: loss: 0.6299 (0.17 sec/step)
INFO:tensorflow:global step 17422: loss: 0.3287 (0.18 sec/step)
INFO:tensorflow:global step 17423: loss: 0.4030 (0.17 sec/step)
INFO:tensorflow:global step 17424: loss: 0.2367 (0.17 sec/step)
INFO:tensorflow:global step 17425: loss: 0.2353 (0.17 sec/step)
INFO:tensorflow:global step 17426: loss: 0.2525 (0.18 sec/step)
INFO:tensorflow:global step 17427: loss: 0.3372 (0.17 sec/step)
INFO:tensorflow:global step 17428: loss: 0.2531 (0.17 sec/step)
INFO:tensorflow:global step 17429: loss:

INFO:tensorflow:global step 17543: loss: 0.2736 (0.17 sec/step)
INFO:tensorflow:global step 17544: loss: 0.2469 (0.17 sec/step)
INFO:tensorflow:global step 17545: loss: 0.2377 (0.17 sec/step)
INFO:tensorflow:global step 17546: loss: 0.2472 (0.17 sec/step)
INFO:tensorflow:global step 17547: loss: 0.2697 (0.18 sec/step)
INFO:tensorflow:global step 17548: loss: 0.2815 (0.17 sec/step)
INFO:tensorflow:global step 17549: loss: 0.2388 (0.18 sec/step)
INFO:tensorflow:global step 17550: loss: 0.2712 (0.18 sec/step)
INFO:tensorflow:global step 17551: loss: 0.2436 (0.17 sec/step)
INFO:tensorflow:global step 17552: loss: 0.2426 (0.17 sec/step)
INFO:tensorflow:global step 17553: loss: 0.4566 (0.17 sec/step)
INFO:tensorflow:global step 17554: loss: 0.2386 (0.18 sec/step)
INFO:tensorflow:global step 17555: loss: 0.2439 (0.17 sec/step)
INFO:tensorflow:global step 17556: loss: 0.3907 (0.18 sec/step)
INFO:tensorflow:global step 17557: loss: 0.2343 (0.17 sec/step)
INFO:tensorflow:global step 17558: loss:

INFO:tensorflow:global step 17664: loss: 0.2552 (0.17 sec/step)
INFO:tensorflow:global step 17665: loss: 0.2360 (0.17 sec/step)
INFO:tensorflow:global step 17666: loss: 0.2939 (0.17 sec/step)
INFO:tensorflow:global step 17667: loss: 0.2355 (0.18 sec/step)
INFO:tensorflow:global step 17668: loss: 0.2550 (0.17 sec/step)
INFO:tensorflow:global step 17669: loss: 0.3002 (0.18 sec/step)
INFO:tensorflow:global step 17670: loss: 0.2695 (0.17 sec/step)
INFO:tensorflow:global step 17671: loss: 0.2379 (0.18 sec/step)
INFO:tensorflow:global step 17672: loss: 0.2812 (0.18 sec/step)
INFO:tensorflow:global step 17673: loss: 0.2957 (0.18 sec/step)
INFO:tensorflow:global step 17674: loss: 0.3502 (0.18 sec/step)
INFO:tensorflow:global step 17675: loss: 0.2520 (0.17 sec/step)
INFO:tensorflow:global step 17676: loss: 0.3584 (0.17 sec/step)
INFO:tensorflow:global step 17677: loss: 0.2797 (0.18 sec/step)
INFO:tensorflow:global step 17678: loss: 0.3547 (0.18 sec/step)
INFO:tensorflow:global step 17679: loss:

INFO:tensorflow:global step 17793: loss: 0.2662 (0.19 sec/step)
INFO:tensorflow:global step 17794: loss: 0.2378 (0.19 sec/step)
INFO:tensorflow:global step 17795: loss: 0.2332 (0.18 sec/step)
INFO:tensorflow:global step 17796: loss: 0.3271 (0.18 sec/step)
INFO:tensorflow:global step 17797: loss: 0.2598 (0.18 sec/step)
INFO:tensorflow:global step 17798: loss: 0.2708 (0.17 sec/step)
INFO:tensorflow:global step 17799: loss: 0.2536 (0.18 sec/step)
INFO:tensorflow:global step 17800: loss: 0.4385 (0.17 sec/step)
INFO:tensorflow:global step 17801: loss: 0.3570 (0.17 sec/step)
INFO:tensorflow:global step 17802: loss: 0.2492 (0.17 sec/step)
INFO:tensorflow:global step 17803: loss: 0.2463 (0.17 sec/step)
INFO:tensorflow:global step 17804: loss: 0.3002 (0.18 sec/step)
INFO:tensorflow:global step 17805: loss: 0.2439 (0.18 sec/step)
INFO:tensorflow:global step 17806: loss: 0.5361 (0.17 sec/step)
INFO:tensorflow:global step 17807: loss: 0.4631 (0.17 sec/step)
INFO:tensorflow:global step 17808: loss:

INFO:tensorflow:Finished training! Saving model to disk now.
